In [2]:
from free_lie_algebra import *
from fla_addendum import *

In [3]:
noGenerators = 2
depth = 3
H = HallBasis(noGenerators, depth, lessExpression=lessExpressionLyndon) # create Lyndon basis
PBW = TensorSpaceBasis(P,H) # lift to the non-symmetrised PBW basis

In [ ]:
""" Output the list of Hall trees, hopefully matching the order of TensorSpaceBasis.data, given a Hall basis lifted to a basis of the tensor
algebra. Either pass it both the basis and lifted basis (to save time), or pass it the Hall basis and function fn words -> tensor basis elements to
use in the lift. """
def hallTreeList(hallBasis, tensorBasis = None, fn = None):
    if tensorBasis == None:
        tensorBasis = TensorSpaceBasis(fn, basis = hallBasis, checkFn=False)
    alphabet=list(range(1,tensorBasis.d+1))
    pbwIndexList = [[]]
    for r in range(1,tensorBasis.m+1):
        words = list(itertools.product(alphabet,repeat=r))
        for w in words:
            pbwIndexList.append(hallBasis.factorIntoHallWords(w))
    return pbwIndexList

print(hallTreeList(H,tensorBasis = PBW))

In [9]:
htl = hallTreeList(H,tensorBasis = PBW)
tensor = word2Elt((1,1,2)) + 3*word2Elt((2,1,2)) - 3*word2Elt((2,2,1)) + 2*word2Elt((2,1))
tcheck = word2Elt(emptyWord)
for i in range(len(htl)):
    r = round(PBW.fromElt(tensor)[i], 4)
    if r != 0:
        print(htl[i], ":    ", r)
        lieList = [basisElementToElt(a) for a in htl[i]]
        tcheck = tcheck + r*concatenationProductMany(lieList)
assert tcheck - tensor - word2Elt(emptyWord) == word2Elt(emptyWord) - word2Elt(emptyWord)

[(2,), (1,)] :     2.0
[((1,), ((1,), (2,)))] :     1.0
[((1,), (2,)), (1,)] :     2.0
[(2,), (1,), (1,)] :     1.0
[(2,), ((1,), (2,))] :     3.0


In [6]:
def removeSmalls(a):
    """a version of an Elt with tiny elements removed"""
    assert isinstance(a,Elt), a
    d=[{k:v for k,v in i.items() if math.fabs(v)>1e-4} for i in a.data] # I chose 4 based on examples
    return Elt(d)

In [7]:
""" Given an Elt, output its non-negligible Hall coordinates as a dictionary  """
def hallDictFromElt(tensor, hallBasis, tensorBasis = None, fn = None, product = None): # should be renamed, since it actually doesn't return a dict...
    if product == None: # product should be concatenationProductMany if fn is P, symConcatProdMany if fn is Psym
        product = concatenationProductMany
    if tensorBasis == None:
        tensorBasis = TensorSpaceBasis(fn, basis = hallBasis, checkFn=False)
    htl = hallTreeList(hallBasis,tensorBasis)
    tcheck = word2Elt(emptyWord)
    for i in range(len(htl)):
        r = round(tensorBasis.fromElt(tensor)[i], 4)
        if r != 0:
            print(htl[i], ":    ", r)
            lieList = [basisElementToElt(a) for a in htl[i]]
            tcheck = tcheck + r*product(lieList)
    print(removeSmalls(tcheck - tensor - word2Elt(emptyWord))) # I will avoid asserting this, since it may still be e-3 or something

In [ ]:
t = word2Elt((1,2,2)) + 3*word2Elt((2,)) - 3*word2Elt((2,1,1)) + 2*word2Elt((2,1)) + 40*word2Elt((2,1,2))
hallDictFromElt(t, H, PBW)

In [78]:
""" Symmetric product, and the mapping of words to symmetric PBW elements """

def symConcatProdMany(a, maxLevel=None):
    f = 1/math.factorial(len(a))
    s = word2Elt(emptyWord)
    for b in list(itertools.permutations(a)): #why doesn't sum() of Elts work
        s = s + concatenationProductMany(b)
    s = s - word2Elt(emptyWord)
    return f*s

def Psym(w, basis):
    assert isinstance(basis, HallBasis), basis
    assert type(w) in (tuple,str), w
    if 0==len(w):
        return unitElt
    assert 0<len(w)<=basis.m
    a=basis.factorIntoHallWords(w)
    h = [basisElementToElt(i) for i in a]
    out = symConcatProdMany(h)
    return out

In [104]:
"""Now let's compute Christian's elements. For the first three I will use 2 generators (ignoring the fact that we have epsilon_0 in one and
epsilon_2 in another, since within each example we only have two generators) and depth 8, for the third I will use 3 generators and depth 6. """
a = symConcatProdMany([word2Elt(Word([1])), word2Elt(Word([2,2,2,2]))])
b = symConcatProdMany([word2Elt(Word([1,1,1,1])), word2Elt(Word([2,2,2,2]))])
c = symConcatProdMany([word2Elt(Word([1,1])), word2Elt(Word([2,2,2,2]))])

d = symConcatProdMany([word2Elt(Word([1,1])), word2Elt(Word([2,2])), word2Elt(Word([3,3]))])

H28 = HallBasis(2, 8, lessExpression=lessExpressionLyndon)
H36 = HallBasis(3, 6, lessExpression=lessExpressionLyndon)
PBW28 = TensorSpaceBasis(Psym,H28,checkFn=False)
PBW36 = TensorSpaceBasis(Psym,H36,checkFn=False)

In [ ]:
hallDictFromElt(a, H28, tensorBasis = PBW28, fn = Psym, product = symConcatProdMany)

In [ ]:
hallDictFromElt(b, H28, tensorBasis = PBW28, fn = Psym, product = symConcatProdMany)

In [ ]:
hallDictFromElt(c, H28, tensorBasis = PBW28, fn = Psym, product = symConcatProdMany)

In [ ]:
hallDictFromElt(d, H36, tensorBasis = PBW36, fn = Psym, product = symConcatProdMany)

In [ ]:
hallDictFromElt(d, H36, fn = P, product = concatenationProductMany)

In [ ]:
""" a check that the last line in the function is not trivial: if the product and the mapping words -> tensor basis elements do not match, you get a
non-zero error """
hallDictFromElt(d, H36, fn = Psym, product = concatenationProductMany)

In [ ]:
# Christian's thesis, Lemma 44 (typo - there is a missing 90)
e = 90*symConcatProdMany([word2Elt(Word([1,1])), word2Elt(Word([2,2])), word2Elt(Word([2,2]))])
hallDictFromElt(e, H28, tensorBasis = PBW28, fn = Psym, product = symConcatProdMany)

In [ ]:
f = 6*symConcatProdMany([word2Elt(Word([1])),word2Elt(Word([1])), word2Elt(Word([2,2]))])
hallDictFromElt(f, H28, tensorBasis = PBW28, fn = Psym, product = symConcatProdMany)

In [ ]:
"""TODO next:
- write code to format properly the Hall tress properly (((1,), (2,)), (2,)), (1,) -> [[1,2],2]*1
- check that the Psym basis is dual to the S_h basis
- compute the expected signature of brownian motion (with drift) in the PSym coordinates up to some level. Does this
help with cubature though?
"""